In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [2]:
import sys
sys.getdefaultencoding()

'utf-8'

Open the questionnaire file (word document version of the questionnaire saved as html):

In [3]:
questionnaire_file = open("FF_Y15_Questionnaire_PCG_2017.htm", "r") 

In [4]:
soup=BeautifulSoup(questionnaire_file, "html.parser")

In [5]:
ps=[p for p in soup.find_all('p')]

In [6]:
ps[108:111]

[<p class="MsoListParagraphCxSpFirst" style="margin-left:45.0pt"><span style='font-size:10.0pt;font-family:"Arial",sans-serif'> </span></p>,
 <p class="MsoListParagraphCxSpLast" style="margin-left:.75in;text-indent:-.25in"><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>AA1.<span style='font:7.0pt "Times New Roman"'>       </span></span><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>Do you receive payment
 for taking care of {YOUTH} or for any other children in your household? Please
 include any foster care payments, or any payments from TANF or welfare, as well
 as financial help from {YOUTH}’s mother or father or from the parents of other
 children living with you.</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} ONLY..................................... 
 1</span></p>]

Opening the excel file with all the questions for p6, using the pandas python library:

In [7]:
df=pd.read_excel("FFCp6.xlsx") #this is an excel file with all the questions for p6

In [8]:
df.head()

,position,name,varlab,probe,exactText
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,NaN
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,NaN,NaN


In [9]:
df.shape

(654, 5)

In [10]:
reg = re.compile(r'(?P<section>[A-Za-z]*\d*)(?P<rest>.*)')

def format_q_num(name):
    """takes in the question numbers in the name column and outputs them in the format
    they appear in the text"""
    name=name.lstrip('p6').split('_')[0] 
    name=reg.search(name)
    section=name.group('section').upper()
    rest=name.group('rest').lower()
    return section+rest

In [11]:
df['question_number']=df.name.apply(lambda x: format_q_num(x))

In [12]:
df['multiple_answers']=df.name.apply(lambda x: 1 if "_" in x else 0) 
#adds a column where 1 indicates that it is one of the questions with multiple entries in the data frame
#I think these are mostly (only?) the "select all that apply" questions

In [13]:
df.head(10)

,position,name,varlab,probe,exactText,question_number,multiple_answers
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN,CONF1,0
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN,CONF2,0
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN,CONF3,0
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,NaN,AA1,0
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,NaN,NaN,AA2,1
5,15556,p6aa2_2,AA2_2. NPCG receives payment from youth's father,NaN,NaN,AA2,1
6,15557,p6aa2_3,AA2_3. NPCG receives payment from parent or pa...,NaN,NaN,AA2,1
7,15558,p6aa2_4,AA2_4. NPCG receives payment from foster care,NaN,NaN,AA2,1
8,15559,p6aa2_5,AA2_5. NPCG receives payment from TANF/welfare,NaN,NaN,AA2,1
9,15560,p6aa2_91,AA2_91. NPCG receives payment from other,NaN,NaN,AA2,1


In [14]:
def find_question(q_number):
    """finds where the question number appears in the text - only if it only appears once"""
    ps_text = []
    ps_idx = []
    ps_startswith = []
    for idx, p in enumerate(ps):
        if q_number+"." in p.get_text():
            if p.get_text().lstrip().startswith(q_number+"."):
                ps_text.append(p.get_text().lstrip())
                ps_idx.append(idx)
    if len(ps_idx)!=1:
        return np.nan, np.nan
    else:
        return ps_text[0].replace("\n"," ").strip(), ps_idx[0]

In [15]:
df['ps_text'], df['ps_idx'] = zip(*df['question_number'].map(find_question))

In [16]:
df

,position,name,varlab,probe,exactText,question_number,multiple_answers,ps_text,ps_idx
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN,CONF1,0,NaN,NaN
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN,CONF2,0,NaN,NaN
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN,CONF3,0,NaN,NaN
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,NaN,AA1,0,AA1. Do you receive payment for taking c...,109.0
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0
5,15556,p6aa2_2,AA2_2. NPCG receives payment from youth's father,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0
6,15557,p6aa2_3,AA2_3. NPCG receives payment from parent or pa...,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0
7,15558,p6aa2_4,AA2_4. NPCG receives payment from foster care,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0
8,15559,p6aa2_5,AA2_5. NPCG receives payment from TANF/welfare,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0
9,15560,p6aa2_91,AA2_91. NPCG receives payment from other,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0


In [17]:
np.isnan(df.ps_idx).sum() #number of questions where the question number doesn't appear or appears more than once

29

In [18]:
print(df.ps_text[11])
print(df.ps_idx[11])

AA3.       How much did you receive altogether last month? Please include payments you received last month for {YOUTH} and for any other children in your household for whom you received payments.
143.0


In [19]:
#get index of <p> tag for the next question so we can split the questions (can't just shift because some questions have more than one answer or missing values)
all_idx=np.array(df.ps_idx)
all_idx=all_idx[~np.isnan(all_idx)]
all_idx=np.append(all_idx, [len(ps)])
df['ps_idx_next']=df.ps_idx.apply(lambda x: all_idx[all_idx > x].min() if ~np.isnan(x) else np.nan)

In [20]:
df

,position,name,varlab,probe,exactText,question_number,multiple_answers,ps_text,ps_idx,ps_idx_next
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN,CONF1,0,NaN,NaN,NaN
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN,CONF2,0,NaN,NaN,NaN
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN,CONF3,0,NaN,NaN,NaN
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,NaN,AA1,0,AA1. Do you receive payment for taking c...,109.0,117.0
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0
5,15556,p6aa2_2,AA2_2. NPCG receives payment from youth's father,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0
6,15557,p6aa2_3,AA2_3. NPCG receives payment from parent or pa...,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0
7,15558,p6aa2_4,AA2_4. NPCG receives payment from foster care,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0
8,15559,p6aa2_5,AA2_5. NPCG receives payment from TANF/welfare,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0
9,15560,p6aa2_91,AA2_91. NPCG receives payment from other,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0


In [21]:
aa1=ps[109:117]

In [22]:
aa1

[<p class="MsoListParagraphCxSpLast" style="margin-left:.75in;text-indent:-.25in"><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>AA1.<span style='font:7.0pt "Times New Roman"'>       </span></span><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>Do you receive payment
 for taking care of {YOUTH} or for any other children in your household? Please
 include any foster care payments, or any payments from TANF or welfare, as well
 as financial help from {YOUTH}’s mother or father or from the parents of other
 children living with you.</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} ONLY..................................... 
 1</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>YES, PAYMENTS FOR OTHER CHILDREN ONLY...................... 
 2</span></p>,
 <p class="MsoNormal" style="margin-left:

In [23]:
for p in aa1:
    print(p)

<p class="MsoListParagraphCxSpLast" style="margin-left:.75in;text-indent:-.25in"><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>AA1.<span style='font:7.0pt "Times New Roman"'>       </span></span><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>Do you receive payment
for taking care of {YOUTH} or for any other children in your household? Please
include any foster care payments, or any payments from TANF or welfare, as well
as financial help from {YOUTH}’s mother or father or from the parents of other
children living with you.</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} ONLY..................................... 
1</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>YES, PAYMENTS FOR OTHER CHILDREN ONLY...................... 
2</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span sty

In [24]:
consequitivedots = re.compile(r'\.{3,}')

In [25]:
for p in aa1:
    if len(p.find_all(string = consequitivedots))>0:
        print(p)

<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} ONLY..................................... 
1</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>YES, PAYMENTS FOR OTHER CHILDREN ONLY...................... 
2</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} AND OTHER CHILDREN......... 
3</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>NO............................................................................................ 
4 </span><span style="font-size:10.0pt;font-family:ZapfDingbats">è</span><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>GO TO Section B</span></p>
<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
font-

In [26]:
answers = [p for p in aa1 if len(p.find_all(string = consequitivedots))>0]

In [27]:
answers

[<p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} ONLY..................................... 
 1</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>YES, PAYMENTS FOR OTHER CHILDREN ONLY...................... 
 2</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>YES, PAYMENTS FOR {YOUTH} AND OTHER CHILDREN......... 
 3</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-size:10.0pt;
 font-family:"Arial",sans-serif'>NO............................................................................................ 
 4 </span><span style="font-size:10.0pt;font-family:ZapfDingbats">è</span><span style='font-size:10.0pt;font-family:"Arial",sans-serif'>GO TO Section B</span></p>,
 <p class="MsoNormal" style="margin-left:.5in"><span style='font-s

In [28]:
print(answers[0].prettify())

<p class="MsoNormal" style="margin-left:.5in">
 <span style='font-size:10.0pt;
font-family:"Arial",sans-serif'>
  YES, PAYMENTS FOR {YOUTH} ONLY..................................... 
1
 </span>
</p>



In [29]:
answers = [p for p in aa1 if len(p.find_all(string = consequitivedots))>0]
for a in answers:
    print(a.get_text().split(".")[0])
    print(a.get_text().split(".")[-1])

YES, PAYMENTS FOR {YOUTH} ONLY
 
1
YES, PAYMENTS FOR OTHER CHILDREN ONLY
 
2
YES, PAYMENTS FOR {YOUTH} AND OTHER CHILDREN
 
3
NO
 
4 èGO TO Section B
REFUSED
 -1èGO TO Section B
DON’T KNOW
 -2èGO TO Section B


In [30]:
def clean_text(string):
    return string.replace("\n"," ").strip()

def get_answers(start_idx,stop_idx):
    if np.isnan(start_idx) or np.isnan(stop_idx):
        return ([],0)
    else:
        answers = [p for p in ps[int(start_idx):int(stop_idx)] if len(p.find_all(string = consequitivedots))>0]
        answers = [(clean_text(a.get_text().split(".")[0]),clean_text(a.get_text().split(".")[-1])) for a in answers]
        return(answers,len(answers)) 

In [31]:
df['answers'], df['n_answers'] = zip(*map(get_answers, df['ps_idx'], df['ps_idx_next']))

In [32]:
x = 0
while x < len(df.answers):
    ans = df.answers[x]
    for i in range(len(ans)): #for each answer choice,
        s = ans[i][1]        # store number string
        a = re.sub('è',' --> ', s) # sub the arrow in for skip pattern
        t = (ans[i][0],a) # create a new tuple with arrow
        ans[i] = t
    for y in range(len(ans)):
        df.loc[x,'val{0}'.format(y)] = ans[y][1] #create new 'val(y)' column
        df.loc[x,'lab{0}'.format(y)] = ans[y][0] #create new 'lab(y)' column
        
    x += 1

In [33]:
df.head(10)

,position,name,varlab,probe,exactText,question_number,multiple_answers,ps_text,ps_idx,ps_idx_next,...,val39,lab39,val40,lab40,val41,lab41,val42,lab42,val43,lab43
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN,CONF1,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN,CONF2,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN,CONF3,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,NaN,AA1,0,AA1. Do you receive payment for taking c...,109.0,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15556,p6aa2_2,AA2_2. NPCG receives payment from youth's father,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15557,p6aa2_3,AA2_3. NPCG receives payment from parent or pa...,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,15558,p6aa2_4,AA2_4. NPCG receives payment from foster care,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15559,p6aa2_5,AA2_5. NPCG receives payment from TANF/welfare,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15560,p6aa2_91,AA2_91. NPCG receives payment from other,NaN,NaN,AA2,1,AA2. Where do you receive payments from?,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.probe

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
       ..
624   NaN
625   NaN
626   NaN
627   NaN
628   NaN
629   NaN
630   NaN
631   NaN
632   NaN
633   NaN
634   NaN
635   NaN
636   NaN
637   NaN
638   NaN
639   NaN
640   NaN
641   NaN
642   NaN
643   NaN
644   NaN
645   NaN
646   NaN
647   NaN
648   NaN
649   NaN
650   NaN
651   NaN
652   NaN
653   NaN
Name: probe, Length: 654, dtype: float64

In [35]:
df.columns.get_loc('val0')

12

In [36]:
i = 0
while i < len(df.ps_text):
    if df.multiple_answers[i] == 1:
        df.loc[i,'probe'] = df.loc[i,'ps_text']
        values = df.loc[i,'val0':].tolist()
        part = (df.name[i].lstrip('p6').split('_')[1])
        match = re.match('[0-9]{1,3}',part)
        if match:
            if part in values:
                index = values.index(part) +1
                text = values[index]
        df.loc[i,'ps_text'] = text
    i += 1

In [37]:
print(df.ps_text[11])

AA3.       How much did you receive altogether last month? Please include payments you received last month for {YOUTH} and for any other children in your household for whom you received payments.


In [38]:
df.answers[11]

[('REFUSED', '-1'), ('DON’T KNOW', '-2')]

In [39]:
df.head(20)

,position,name,varlab,probe,exactText,question_number,multiple_answers,ps_text,ps_idx,ps_idx_next,...,val39,lab39,val40,lab40,val41,lab41,val42,lab42,val43,lab43
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN,CONF1,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN,CONF2,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN,CONF3,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,NaN,AA1,0,AA1. Do you receive payment for taking c...,109.0,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,AA2. Where do you receive payments from?,NaN,AA2,1,YOUTH’S MOTHER,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15556,p6aa2_2,AA2_2. NPCG receives payment from youth's father,AA2. Where do you receive payments from?,NaN,AA2,1,YOUTH’S FATHER,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15557,p6aa2_3,AA2_3. NPCG receives payment from parent or pa...,AA2. Where do you receive payments from?,NaN,AA2,1,PARENT OR PARENTS OF OTHER CHILDREN,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,15558,p6aa2_4,AA2_4. NPCG receives payment from foster care,AA2. Where do you receive payments from?,NaN,AA2,1,FOSTER CARE PAYMENT,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15559,p6aa2_5,AA2_5. NPCG receives payment from TANF/welfare,AA2. Where do you receive payments from?,NaN,AA2,1,TANF/WELFARE,117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15560,p6aa2_91,AA2_91. NPCG receives payment from other,AA2. Where do you receive payments from?,NaN,AA2,1,OTHER (SPECIFY),117.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.drop(['answers','n_answers','multiple_answers','exactText','question_number','ps_idx','ps_idx_next'],axis=1,inplace=True)

In [41]:
df.head(20)

,position,name,varlab,probe,ps_text,val0,lab0,val1,lab1,val2,...,val39,lab39,val40,lab40,val41,lab41,val42,lab42,val43,lab43
0,15551,p6conf1,CONF1. PCG relationship to youth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15552,p6conf2,CONF2. How much time youth lives with PCG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15553,p6conf3,CONF3. Other person youth usually lives with,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15554,p6aa1,AA1. NPCG receives payment for caring for youth?,NaN,AA1. Do you receive payment for taking c...,1,"YES, PAYMENTS FOR {YOUTH} ONLY",2,"YES, PAYMENTS FOR OTHER CHILDREN ONLY",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15555,p6aa2_1,AA2_1. NPCG receives payment from youth's mother,AA2. Where do you receive payments from?,YOUTH’S MOTHER,1,YOUTH’S MOTHER,2,YOUTH’S FATHER,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15556,p6aa2_2,AA2_2. NPCG receives payment from youth's father,AA2. Where do you receive payments from?,YOUTH’S FATHER,1,YOUTH’S MOTHER,2,YOUTH’S FATHER,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15557,p6aa2_3,AA2_3. NPCG receives payment from parent or pa...,AA2. Where do you receive payments from?,PARENT OR PARENTS OF OTHER CHILDREN,1,YOUTH’S MOTHER,2,YOUTH’S FATHER,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,15558,p6aa2_4,AA2_4. NPCG receives payment from foster care,AA2. Where do you receive payments from?,FOSTER CARE PAYMENT,1,YOUTH’S MOTHER,2,YOUTH’S FATHER,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15559,p6aa2_5,AA2_5. NPCG receives payment from TANF/welfare,AA2. Where do you receive payments from?,TANF/WELFARE,1,YOUTH’S MOTHER,2,YOUTH’S FATHER,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15560,p6aa2_91,AA2_91. NPCG receives payment from other,AA2. Where do you receive payments from?,OTHER (SPECIFY),1,YOUTH’S MOTHER,2,YOUTH’S FATHER,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df.to_csv(path_or_buf = "p6Qtext.csv")